In [22]:
import pandas as pd
from openai import AzureOpenAI
from tqdm import tqdm
import os

# 1. Configuración de OpenAI
client = AzureOpenAI(
    api_key="4b6sN2Bj3FAPlNnbhVqtEjXMGhoTS69lfAXKLyDM9sotWzDxp3bjJQQJ99BEACYeBjFXJ3w3AAABACOGjsn0",
    api_version="2023-05-15",
    azure_endpoint="https://voiceflip-openai.openai.azure.com/"
)

# 2. Cargar datos
df_segments = pd.read_csv('D:/voiceflip/notebooks/data/processed/churn_segmented.csv')

# 3. Función para generar mensajes personalizados
def generate_message(metrics):
    risk_descriptions = {
        1: "cliente fiel con bajo riesgo",
        2: "cliente con algunas señales de riesgo",
        3: "cliente que requiere atención inmediata"
    }
    risk_text = risk_descriptions.get(metrics['risk_group'], "cliente")
    
    prompt = f"""
    Genera un mensaje profesional y personalizado en español para un {risk_text} con:
    - Nivel de engagement actual: {metrics['engagement']:.2f}
    - Probabilidad de abandono: {metrics['churn_probability']:.2f}
    
    El mensaje debe:
    1. Ser formal y directo
    2. Mencionar las métricas específicas de uso
    3. Incluir 2 recomendaciones concretas de productos VoiceFlip:
    4. Terminar con una llamada a la acción clara
    5. Incluir firma profesional
    
    No uses placeholders ni variables. Todo debe ser específico.
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # Corregido: gpt-4 en lugar de gpt-4o
            messages=[
                {
                    "role": "system", 
                    "content": "Eres un especialista en retención de clientes de VoiceFlip, una plataforma de procesamiento de voz y análisis de audio."
                },
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error generando mensaje: {e}"

# 4. Procesar usuarios
results = []
for _, row in tqdm(df_segments.head(100).iterrows(), desc="Generando mensajes"):
    try:
        msg = generate_message({
            "risk_group": row["risk_group"],
            "engagement": row["engagement"],
            "churn_probability": row["churn_probability"]
        })
        results.append({
            "user_id": row.name,
            "risk_group": row["risk_group"],
            "churn_prob": row["churn_probability"],
            "message": msg
        })
    except Exception as e:
        print(f"Error con usuario {row.name}: {str(e)}")

# 5. Crear DataFrame con resultados
messages_df = pd.DataFrame(results)



# 7. Guardar resultados
output_path = 'output/personalized_messages.csv'
os.makedirs('output', exist_ok=True)
messages_df.to_csv(output_path, index=False)
print(f"\nMensajes guardados en: {output_path}")

Generando mensajes: 100it [06:43,  4.03s/it]


Mensajes guardados en: output/personalized_messages.csv
